# Parse the XML file for HQTA files

* [Edit metadata for many ArcGIS items](https://desktop.arcgis.com/en/arcmap/latest/manage-data/metadata/editing-metadata-for-many-arcgis-items.htm)
* [Change to JSON](https://stackoverflow.com/questions/48821725/xml-parsers-expat-expaterror-not-well-formed-invalid-token)

In [ ]:
#!pip install xmltodict

In [1]:
import xml.etree.ElementTree as ET
import json
import xmltodict

In [2]:
xml_file = "./data/ca_hq_transit_areas.xml"

In [3]:
# read in XML
tree = ET.parse(xml_file)
root = tree.getroot()

In [28]:
# Not well-formed XML?
# Figure out another way to read in as a dict
#for elem in tree.iter():
#    print(elem.tag, elem.attrib)

metadata {}
idinfo {}
citation {}
citeinfo {}
title {}
geoform {}
pubinfo {}
publish {}
descript {}
abstract {}
purpose {}
timeperd {}
timeinfo {}
rngdates {}
begdate {}
enddate {}
current {}
status {}
progress {}
update {}
spdom {}
bounding {}
westbc {}
eastbc {}
northbc {}
southbc {}
keywords {}
theme {}
themekt {}
themekey {}
theme {}
themekt {}
themekey {}
themekey {}
themekey {}
themekey {}
themekey {}
accconst {}
useconst {}
native {}
dataqual {}
lineage {}
procstep {}
procdesc {}
spdoinfo {}
direct {}
ptvctinf {}
sdtsterm {}
sdtstype {}
ptvctcnt {}
eainfo {}
detailed {}
enttyp {}
enttypl {}
attr {}
attrlabl {}
attrdef {}
attrdefs {}
attrdomv {}
udom {}
attr {}
attrlabl {}
attrdef {}
attrdefs {}
attrdomv {}
udom {}
attr {}
attrlabl {}
attr {}
attrlabl {}
attr {}
attrlabl {}
attr {}
attrlabl {}
attr {}
attrlabl {}
attr {}
attrlabl {}
attrdef {}
attrdefs {}
attrdomv {}
udom {}
attr {}
attrlabl {}
attrdef {}
attrdefs {}
attrdomv {}
udom {}
detailed {}
enttyp {}
enttypd {}
enttypds {

In [50]:
def xml_to_json(path: str) -> dict:  
    try:
        print(f"Loading XML as JSON from {path}")
        xml = ET.tostring(ET.parse(path).getroot())
        return xmltodict.parse(xml, 
                               #attr_prefix="@", cdata_key="#text", 
                               dict_constructor=dict)
    except:
        print(f"Loading failed for {path}")
    return {}

metadata_json = xml_to_json(xml_file)

Loading XML as JSON from ./data/ca_hq_transit_areas.xml


In [75]:
keyword_list = metadata_json["metadata"]["idinfo"]["keywords"]["theme"]

[{'themekt': 'ISO 19115 Topic Categories', 'themekey': 'transportation'},
 {'themekt': 'None',
  'themekey': ['Transportation',
   'Land Use',
   'Transit-Oriented Development',
   'TOD',
   'High Quality Transit']}]

In [79]:
def fill_in_keyword_list(topic='transportation', keyword_list = []):
    filled_out_list = [
        {'themekt': 'ISO 19115 Topic Categories',
         'themekey': topic},
         {'themekt': 'None',
          'themekey': keyword_list
         }
    ]
    
    return filled_out_list

In [80]:
theme_keywords = ["One", "Two", "Three", "Four", "Five"]
fill_in_keyword_list("transportation", theme_keywords)

[{'themekt': 'ISO 19115 Topic Categories', 'themekey': 'transportation'},
 {'themekt': 'None', 'themekey': ['One', 'Two', 'Three', 'Four', 'Five']}]

In [ ]:
 metadata_json["metadata"]["idinfo"]["keywords"]["theme"]

In [57]:
DATASET_INFO = {
    ## my_json["metadata"]["idinfo"]
    # ["citation"]
    "dataset_name": "ca_hq_transit_areas", #["citeinfo"]["title"]
    "publish_entity": "California Integrated Travel Project", #["pubinfo"]["publish"]
    # ["descript"]
    "abstract": "Public. EPSG: 3310", #["abstract"]
    "purpose": "Summary sentence about dataset.", #["purpose"]
    # ["timeperd"]
    "beginning_date": "YYYYMMDD", #["timeinfo"]["rngdates"]["begdate"]
    "end_date": "YYYYMMDD", #["timeinfo"]["rngdates"]["enddate"]
    "place": "California", #["current"]
    # ["status"]
    "status": "Complete", #["progress"]
    "frequency": "Monthly", #["update"]
    # ["keywords"]
    # transportation is probably what we always want to check off
    # Be able to input list (5 is min)...just unpack the phrases and make list
    "theme_topics": fill_in_keyword_list(topic = "transportation", 
                                         theme_keywords = []),
    ## my_json["metadata"]["dataqual"]
    "methodology": "Detailed methodology description", #["lineage"]["procstep"]["procdesc"]
    ## my_json["metadata"]["eainfo"]
    # ["detailed"] (this outputs a list)
    ## set this equal to dataset_name above ["detailed"][0]["enttyp"]["enttypl"] 
    "data_dict_type": "CSV", # ["detailed"][1]["enttyp"]["enttypd"],
    "data_dict_url": "some_url", 
    ## my_json["metadata"]["metainfo"]
    #["metc"]
    "contact_organization": "Caltrans", #["metc"]["cntinfo"]["cntorgp"]["cntorg"]
    "contact_person": "Analyst Name", #["metc"]["cntinfo"]["cntorgp"]["cntper"]
    "contact_position": "California Integrated Travel Project", #["metc"]["cntinfo"]["cntpos"]
    "contact_email": "hello@calitp.org" #["metc"]["cntinfo"]["cntemail"],
}

SyntaxError: ':' expected after dictionary key (2595613434.py, line 4)

In [95]:
metadata_json

{'metadata': {'idinfo': {'citation': {'citeinfo': {'title': 'ca_hq_transit_areas',
     'geoform': 'vector digital data',
     'pubinfo': {'publish': 'California Integrated Travel Project'}}},
   'descript': {'abstract': 'Public. EPSG: 3310',
    'purpose': 'Estimated High Quality Transit Areas as described in Public Resources Code 21155, 21064.3, 21060.2'},
   'timeperd': {'timeinfo': {'rngdates': {'begdate': '20220517',
      'enddate': '20220617'}},
    'current': 'California'},
   'status': {'progress': 'Complete', 'update': 'Monthly'},
   'spdom': {'bounding': {'westbc': '-124.423914',
     'eastbc': '-113.465232',
     'northbc': '45.362934',
     'southbc': '32.441061'}},
   'keywords': {'theme': [{'themekt': 'ISO 19115 Topic Categories',
      'themekey': 'transportation'},
     {'themekt': 'None',
      'themekey': ['Transportation',
       'Land Use',
       'Transit-Oriented Development',
       'TOD',
       'High Quality Transit']}]},
   'accconst': 'None',
   'useconst': 

In [92]:
#metadata_json["metadata"]["eainfo"]["detailed"][1]

In [94]:
#for k, v in metadata_json["metadata"]["metainfo"].items():
#    print(f"Key: {k}")
#    print(v)        

Key: metd
20220602
Key: metc
{'cntinfo': {'cntorgp': {'cntorg': 'Caltrans', 'cntper': 'Eric Dasmalchi'}, 'cntpos': 'California Integrated Travel Project', 'cntemail': 'eric.dasmalchi@dot.ca.gov'}}
Key: metstdn
FGDC Content Standard for Digital Geospatial Metadata
Key: metstdv
FGDC-STD-001-1998
Key: mettc
local time


In [49]:
for key, value in metadata_json["metadata"].items():
    #print(key, value)
    if key == "idinfo":
        for k, v in metadata_json["metadata"][key].items():
            print(v)

{'citeinfo': {'title': 'ca_hq_transit_areas', 'geoform': 'vector digital data', 'pubinfo': {'publish': 'California Integrated Travel Project'}}}
{'abstract': 'Public. EPSG: 3310', 'purpose': 'Estimated High Quality Transit Areas as described in Public Resources Code 21155, 21064.3, 21060.2'}
{'timeinfo': {'rngdates': {'begdate': '20220517', 'enddate': '20220617'}}, 'current': 'California'}
{'progress': 'Complete', 'update': 'Monthly'}
{'bounding': {'westbc': '-124.423914', 'eastbc': '-113.465232', 'northbc': '45.362934', 'southbc': '32.441061'}}
{'theme': [{'themekt': 'ISO 19115 Topic Categories', 'themekey': 'transportation'}, {'themekt': 'None', 'themekey': ['Transportation', 'Land Use', 'Transit-Oriented Development', 'TOD', 'High Quality Transit']}]}
None
None
Version 6.2 (Build 9200) ; Esri ArcGIS 10.8.1.14362
